In [8]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50, DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os
import numpy as np

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [11]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\junu\.cache\kagglehub\datasets\vipoooool\new-plant-diseases-dataset\versions\2


In [12]:
train = 'C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\train'
valid = 'C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\valid'

In [13]:
# Load and Preprocess Dataset
# dataset_path = path  # Path from kagglehub download
image_size = (75,75)  # Standard size for most models
batch_size = 16

# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
    )

# Training and validation generators
train_generator = datagen.flow_from_directory(
    train,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
# Add .prefetch(tf.data.AUTOTUNE) if using tf.data pipeline

val_generator = datagen.flow_from_directory(
    valid,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
    )

Found 56251 images belonging to 38 classes.
Found 3503 images belonging to 38 classes.
Found 3503 images belonging to 38 classes.


In [14]:
# Define Models
def create_model(base_model):
    base_model.trainable = False  # Freeze the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize models
models = {
    'VGG16': create_model(VGG16(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'VGG19': create_model(VGG19(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'InceptionV3': create_model(InceptionV3(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'Xception': create_model(Xception(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'ResNet50': create_model(ResNet50(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'DenseNet121': create_model(DenseNet121(weights='imagenet', include_top=False, input_shape=image_size + (3,)))
}

In [15]:
# Train Models
history = {}
epochs = 5

for model_name, model in models.items():
    print(f"Training {model_name}...")
    history[model_name] = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs
    )
    model.save(f"{model_name}_plant_disease_model.h5")
    print(f"{model_name} training completed and model saved.")

Training VGG16...
Epoch 1/5
Epoch 1/5
 177/3516 ━━━━━━━━━━━━━━━━━━━━ 35:47 643ms/step - accuracy: 0.0579 - loss: 3.6695

KeyboardInterrupt: 

In [ ]:
import numpy as np

# --- Set up the generator for prediction ---
# If using 'val_generator' from training:
# 1. Reset the generator to ensure it starts from the beginning.
val_generator.reset()

# 2. Determine the number of steps required to cover the entire validation set.
# Total samples / Batch size (using ceiling to catch any partial last batch)
steps = np.ceil(val_generator.samples / val_generator.batch_size)

# --- Generate Predictions ---
# y_pred will contain the predicted probabilities (e.g., shape (N_samples, N_classes))
print(f"Generating predictions for {model_name}...")
y_pred = model.predict(val_generator, steps=steps)

# --- Extract True Labels ---
# y_true are the actual class indices (labels) (e.g., shape (N_samples,))
y_true = val_generator.classes[:val_generator.samples]

# --- Save to Disk ---
# Save the numpy arrays so you can load them later.
np.save(f"{model_name}_y_pred.npy", y_pred)
np.save(f"{model_name}_y_true.npy", y_true)

print(f"Predictions and true labels saved for {model_name}.")

In [ ]:
# Generate Model Comparison Table
import numpy as np
import pandas as pd

models = ["vgg16", "vgg19", "inception", "xception", "resnet", "densenet"]
results = []

for name in models:
    y_pred = np.load(f"{name}_y_pred.npy")
    y_true = np.load(f"{name}_y_true.npy")
    y_pred_labels = y_pred.argmax(axis=1)
    accuracy = np.mean(y_pred_labels == y_true)
    results.append({
        'Model': name.upper(),
        'Accuracy': accuracy
    })

comparison_df = pd.DataFrame(results)
print(comparison_df)

# Generate Classification Reports
from sklearn.metrics import classification_report

for name in models:
    y_pred = np.load(f"{name}_y_pred.npy")
    y_true = np.load(f"{name}_y_true.npy")
    y_pred_labels = y_pred.argmax(axis=1)
    print(f"\n===== {name.upper()} =====")
    print(classification_report(y_true, y_pred_labels))

FileNotFoundError: [Errno 2] No such file or directory: 'vgg16_y_pred.npy'